In [6]:
dict = Dict{String, Tuple{Int, Int}}()

Dict{String,Tuple{Int64,Int64}} with 0 entries

In [11]:
struct Test
    x::Int
    y::Float64
end

ErrorException: invalid redefinition of constant Test

In [8]:
test = Test(1,2)

Test(1, 2)

In [9]:
test

Test(1, 2)

In [12]:
struct Test2
    x::Int
    y::Float64
end

In [13]:
test = Test2(1,1)

Test2(1, 1.0)

In [15]:
using Base.Threads

In [16]:
val = Threads.Atomic{Int}(3)

Atomic{Int64}(3)

In [18]:
val[]

3

In [19]:
using DataFrames

┌ Info: Precompiling DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0]
└ @ Base loading.jl:1260


In [20]:
df = DataFrame

DataFrame